# Imports

In [1]:
import torch
import transformers
from tokenizers import Tokenizer
from tokenizers import normalizers, pre_tokenizers, models, processors, decoders, trainers
from torch.utils.data import DataLoader
from datasets import load_dataset, total_allocated_bytes

import config

# Hyper parameters

In [2]:
# Load wikitext-103 dataset
train_ds, val_ds, test_ds = load_dataset('wikitext', 'wikitext-103-raw-v1', split=["train", "validation", "test"])

Reusing dataset wikitext (C:\Users\Marc Bøg\.cache\huggingface\datasets\wikitext\wikitext-103-raw-v1\1.0.0\47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91)


In [3]:
# Tokenizer Configuration - Uses Byte Pair Encoding
# NFKD Unicode Normalization, all lowercase
# Split on whitespace, store info about space (metaspace)
# Split digits from words
trainer = trainers.BpeTrainer(
    vocab_size=config.VOCAB_SIZE,
    special_tokens=["[PAD]", "[CLS]", "[SEP]", "[EOS]", "[UNK]", "[MASK]"],
    show_progress=True
)

tokenizer = Tokenizer(models.BPE())
tokenizer.normalizer = normalizers.Sequence([
    normalizers.NFKD(),
    normalizers.Lowercase(),
    #normalizers.Strip("both"),
])
tokenizer.pre_tokenizer = pre_tokenizers.Sequence([
    pre_tokenizers.Whitespace(),
    #pre_tokenizers.Metaspace(),
    pre_tokenizers.Digits(individual_digits=False)
])
tokenizer.post_processor = processors.TemplateProcessing(
    single="[CLS] $A [EOS]",
    pair="[CLS] $A [SEP] $B:1 [EOS]:1",
    special_tokens=[("[CLS]", 1), ("[SEP]", 2), ("[EOS]", 3)]
)
tokenizer.decoder = decoders.BPEDecoder()



In [4]:
tokenizer.train(trainer, config.PATH_DATA_FILES)

In [5]:
files = tokenizer.model.save(config.PATH_DATA)
tokenizer.model = models.BPE.from_file(*files, unk_token="[UNK]")
tokenizer.save(config.PATH_TOKENIZER)

# Pad to longest string in batch
tokenizer.enable_padding(
    direction="right",
    #length=config.PADDING_SIZE,
    pad_id=tokenizer.token_to_id("[PAD]")
)
# tokenizer.enable_truncation(
#     max_length=config.PADDING_SIZE
# )

In [6]:
def tokenize(batch):
    encoded = tokenizer.encode_batch(batch["text"])
    ids = []
    type_ids = []
    attention_mask = []
    special_tokens_mask = []

    for x in encoded:
        ids.append(x.ids)
        #type_ids.append(x.type_ids)
        attention_mask.append(x.attention_mask)
        #special_tokens_mask.append(x.special_tokens_mask)
    
    return {
        "ids": ids,
        #"type_ids": type_ids,
        "attention_mask": attention_mask,
        #"special_tokens_mask": special_tokens_mask
    }


ttrain_ds = train_ds.map(tokenize, batched=True, batch_size=config.BATCH_SIZE)
tval_ds = val_ds.map(tokenize, batched=True, batch_size=config.BATCH_SIZE)
ttest_ds = test_ds.map(tokenize, batched=True, batch_size=config.BATCH_SIZE)

Loading cached processed dataset at C:\Users\Marc Bøg\.cache\huggingface\datasets\wikitext\wikitext-103-raw-v1\1.0.0\47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91\cache-0fce5efe474d6a5f.arrow
Loading cached processed dataset at C:\Users\Marc Bøg\.cache\huggingface\datasets\wikitext\wikitext-103-raw-v1\1.0.0\47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91\cache-c6b66b3ea0176180.arrow
Loading cached processed dataset at C:\Users\Marc Bøg\.cache\huggingface\datasets\wikitext\wikitext-103-raw-v1\1.0.0\47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91\cache-e946d5fd9bbe9eee.arrow


In [7]:
ttrain_ds.save_to_disk(config.PATH_TRAIN_TOK)
tval_ds.save_to_disk(config.PATH_VAL_TOK)
ttest_ds.save_to_disk(config.PATH_TEST_TOK)

tval_ds.column_names

['attention_mask', 'ids', 'text']

In [8]:
ttrain_ds.set_format(type="pt", columns=["ids", "attention_mask"])

In [9]:
ttrain_ds.column_names

['attention_mask', 'ids', 'text']

In [10]:
dataloader = DataLoader(ttrain_ds, batch_size=config.BATCH_SIZE)

In [11]:
d = iter(dataloader)
next(d)

{'attention_mask': tensor([[1, 1, 0,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 0,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 0,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'ids': tensor([[   1,    3,    0,  ...,    0,    0,    0],
         [   1,   34, 4934,  ...,    0,    0,    0],
         [   1,    3,    0,  ...,    0,    0,    0],
         ...,
         [   1,   34,   34,  ...,    0,    0,    0],
         [   1,    3,    0,  ...,    0,    0,    0],
         [   1, 4162, 5399,  ...,    0,    0,    0]])}

In [13]:
tokenizer.encode_batch(train_ds[0:config.BATCH_SIZE]["text"])

[Encoding(num_tokens=360, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=360, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=360, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=360, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=360, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=360, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=360, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=360, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=360